In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

input_shape  = (224,224,3)

import glob
import pickle
import random
import pydicom

import pandas as pd
import numpy as np
from tqdm import tqdm
from numba import jit, prange

import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns

import tensorflow as tf
print(tf.__version__)

import tensorflow.keras as keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, concatenate, Input
from scipy.spatial import distance
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
#from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.layers import Conv2D, GlobalMaxPooling2D, Conv3D, MaxPool3D, BatchNormalization, Dropout, GlobalAveragePooling3D, Flatten, LeakyReLU
from tensorflow.keras.initializers import HeUniform
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam

from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE

import os
import cv2
from pydicom.pixel_data_handlers.util import apply_voi_lut


import time
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import re
# from gluoncv.utils import viz
# import mxnet as mx
import multiprocessing as mp

# Input data files are available in the read-only "../input/" directory




In [ ]:
! dir '../input/rsna-miccai-brain-tumor-radiogenomic-classification/'

root_dir = '../input/rsna-miccai-brain-tumor-radiogenomic-classification/'
df_train = pd.read_csv(root_dir+'train_labels.csv')
sns.countplot(data=df_train, x='MGMT_value')

In [ ]:
# Add the full paths for each id for different types of sequences to the csv 
def full_ids(data):
    zeros = 5 - len(str(data))
    if zeros > 0:
        prefix = ''.join(['0' for i in range(zeros)])
    
    return prefix+str(data)
        

df_train['PatientID'] = df_train['BraTS21ID'].apply(full_ids)

# Add all the paths to the df for easy access
df_train['flair'] = df_train['PatientID'].apply(lambda file_id : root_dir+'train/'+file_id+'/FLAIR/')
df_train['t1w']   = df_train['PatientID'].apply(lambda file_id : root_dir+'train/'+file_id+'/T1w/')
df_train['t1wce'] = df_train['PatientID'].apply(lambda file_id : root_dir+'train/'+file_id+'/T1wCE/')
df_train['t2w']   = df_train['PatientID'].apply(lambda file_id : root_dir+'train/'+file_id+'/T2w/')

df_train = df_train.drop(['BraTS21ID'], axis=1)
# Delete column
df_train

In [ ]:
def _dicom2array(path, voi_lut=True, fix_monochrome=True, resize=True, img_size=256):
    dicom = pydicom.read_file(path)
    data  = apply_voi_lut(dicom.pixel_array, dicom)
    if dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
    #Normalize the data: subtract off the minimum, divide by the maximum, convert to 256 uint8
    data = data - np.min(data)
    data = data/np.max(data)
    data = (data * 255).astype(np.uint8)
    
    #Resize images to target value
    data = cv2.resize(data, (img_size, img_size))
    data = cv2.equalizeHist(data)
    return data


def _circumscriber(img: np.array) -> np.array:
    #First is vertical, second is horizontal, third is slices
    vmin = 0
    vlimit = img.shape[0]
    hmin = 0
    hlimit = img.shape[1]
    
    for i in range(vlimit):
        if np.max(img[i, :]) == 0:
            vmin += 1
        else:
            break
    vmax = vmin + 1
    for i in range(vmin+1, vlimit):
        if np.max(img[i, :]) > 0:
            vmax += 1
        else:
            break
    
    for j in range(hlimit):
        if np.max(img[:, j]) == 0:
            hmin += 1
        else:
            break
    hmax = hmin + 1
    for j in range(hmin+1, hlimit):
        if np.max(img[:, j]) > 0:
            hmax += 1
        else:
            break
    return img[vmin:vmax, hmin:hmax]

def crop_image(img,tol=100):
    # img is 2D image data
    # tol  is tolerance
    mask = img>tol
    return img[np.ix_(mask.any(1),mask.any(0))]

import operator
def cropND(img, bounding):
    start = tuple(map(lambda a, da: a//2-da//2, img.shape, bounding))
    end   = tuple(map(operator.add, start, bounding))
    slices = tuple(map(slice, start, end))
    return img[slices]


dcm_path = '../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00014/T1w/Image-190.dcm'
#print(data)
img1  = _dicom2array(dcm_path)
print(img1.shape)

#print(img.shape)

fig = plt.figure(figsize=(128, 128))

fig.add_subplot(15, 15, 1)
plt.imshow(img1, cmap="bone")



In [ ]:
import scipy

train_path = '/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification'


def save_slices(root_dir, patient, modality, mgmt, slices):
    filename = '{}\\{}_{}_{}.pickle'.format(root_dir, patient, modality, mgmt)
    with open(filename, 'wb') as handle:
        pickle.dump(slices, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    
    
def load_slices(patient, modality, mgmt):
    filename = '{}\\{}_{}_{}.pickle'.format(root_dir, patient, modality, mgmt)
    with open(filename, 'rb') as handle:
        slices = pickle.load(handle)
    return slices  



def get_ordered_slices(path:str):
    """
    get a list of ordered slices
    """
    images = [os.path.basename(f) for f in glob.glob(f'{path}/*.dcm')]
    images.sort(key=lambda f: int(re.sub('\D', '', f)))
    return images
      

def get_ordered_slices_restrict(path:str):
    """
    get a list of ordered slices
    """
    images   = [os.path.basename(f) for f in glob.glob(f'{path}/*.dcm')]
    images.sort(key=lambda f: int(re.sub('\D', '', f)))
    len_imgs = len(images)
    fourth   = len_imgs//4
    images   = images[fourth:-fourth]
   
    return images    
        



def load_FULL_brain_restricted(dcm_path):
    """
    send all of the images in the chosen modality, in order, as a single 3D np array
    """
    
    images = get_ordered_slices(dcm_path)
    real_images = [_dicom2array(dcm_path + f) for f in images]
    # np.count_nonzero(anchor) < 10000)
    good_images = np.array([im for im in real_images if (np.count_nonzero(im) > 7000)]).T
    
    #final_image = _circumscriber(good_images)
    return good_images


def load_FULL_brain(dcm_path):
    """
    send all of the images in the chosen modality, in order, as a single 3D np array
    """
    
    images = get_ordered_slices(dcm_path)
    real_images = [_dicom2array(dcm_path + f) for f in images]
    # np.count_nonzero(anchor) < 10000)
    good_images = np.array([im for im in real_images if (np.max(im) > 0)]).T
    #final_image = _circumscriber(good_images)
    return good_images


def get_nr_slices(path: str):
    """
    return nr of slices
    """
    return len(glob.glob(f'{path}/*.dcm'))
    
# Old implementaiton   
# def get_ordered_slice(path:str, index:int):
#     """
#     get specific slice
#     """
#     images = [os.path.basename(f) for f in glob.glob(f'{path}/*.dcm')]
#     images.sort(key=lambda f: int(re.sub('\D', '', f)))
#     dicom_path = "{}{}".format(path, images[index])
#     return _dicom2array(dicom_path)



def get_ordered_slice(slice_arr:str, slice_path:int, slice_idx: int):
    """
    get specific slice
    """
    #print(slice_arr[slice_idx])
    #print(slice_path)
   
    #print("-------------")
    slice_file      = slice_arr[slice_idx]
    slice_fullpath  = "{}{}".format(slice_path, slice_file)
    return np.array(_dicom2array(slice_fullpath))


"""
'Accent', 'Accent_r', 'Blues', 'Blues_r', 'BrBG', 'BrBG_r', 'BuGn', 'BuGn_r', 'BuPu', 'BuPu_r', 'CMRmap', 'CMRmap_r', 'Dark2', 'Dark2_r', 'GnBu', 'GnBu_r', 'Greens', 'Greens_r', 'Greys', 'Greys_r', 'OrRd', 'OrRd_r', 'Oranges', 'Oranges_r', 'PRGn', 'PRGn_r', 'Paired', 'Paired_r', 'Pastel1', 'Pastel1_r', 'Pastel2', 'Pastel2_r', 'PiYG', 'PiYG_r', 'PuBu', 'PuBuGn', 'PuBuGn_r', 'PuBu_r', 'PuOr', 'PuOr_r', 'PuRd', 'PuRd_r', 'Purples', 'Purples_r', 'RdBu', 'RdBu_r', 'RdGy', 'RdGy_r', 'RdPu', 'RdPu_r', 'RdYlBu', 'RdYlBu_r', 'RdYlGn', 'RdYlGn_r', 'Reds', 'Reds_r', 'Set1', 'Set1_r', 'Set2', 'Set2_r', 'Set3', 'Set3_r', 'Spectral', 'Spectral_r', 'Wistia', 'Wistia_r', 'YlGn', 'YlGnBu', 'YlGnBu_r', 'YlGn_r', 'YlOrBr', 'YlOrBr_r', 'YlOrRd', 'YlOrRd_r', 'afmhot', 'afmhot_r', 'autumn', 'autumn_r', 'binary', 'binary_r', 'bone', 'bone_r', 'brg', 'brg_r', 'bwr', 'bwr_r', 'cividis', 'cividis_r', 'cool', 'cool_r', 'coolwarm', 'coolwarm_r', 'copper', 'copper_r', 'crest', 'crest_r', 'cubehelix', 'cubehelix_r', 'flag', 'flag_r', 'flare', 'flare_r', 'gist_earth', 'gist_earth_r', 'gist_gray', 'gist_gray_r', 'gist_heat', 'gist_heat_r', 'gist_ncar', 'gist_ncar_r', 'gist_rainbow', 'gist_rainbow_r', 'gist_stern', 'gist_stern_r', 'gist_yarg', 'gist_yarg_r', 'gnuplot', 'gnuplot2', 'gnuplot2_r', 'gnuplot_r', 'gray', 'gray_r', 'hot', 'hot_r', 'hsv', 'hsv_r', 'icefire', 'icefire_r', 'inferno', 'inferno_r', 'jet', 'jet_r', 'magma', 'magma_r', 'mako', 'mako_r', 'nipy_spectral', 'nipy_spectral_r', 'ocean', 'ocean_r', 'pink', 'pink_r', 'plasma', 'plasma_r', 'prism', 'prism_r', 'rainbow', 'rainbow_r', 'rocket', 'rocket_r', 'seismic', 'seismic_r', 'spring', 'spring_r', 'summer', 'summer_r', 'tab10', 'tab10_r', 'tab20', 'tab20_r', 'tab20b', 'tab20b_r', 'tab20c', 'tab20c_r', 'terrain', 'terrain_r', 'turbo', 'turbo_r', 'twilight', 'twilight_r', 'twilight_shifted', 'twilight_shifted_r', 'viridis', 'viridis_r', 'vlag', 'vlag_r', 'winter', 'winter_r'
"""

def plot_imgs(imgs, cols=4, size=7, is_rgb=True, title="", cmap='nipy_spectral', img_size=(224,224)):
    rows = imgs.shape[2]//cols + 2
    fig = plt.figure(figsize=(cols*size, rows*size))
    for i in range(imgs.shape[2]-1):
        img = imgs[:,:,i]
        fig.add_subplot(rows, cols, i+1)
        plt.imshow(img)
    plt.suptitle(title)
    plt.show()


    
def plot_img_list(imgs, cols=4, size=7, is_rgb=True, title="", cmap='bone', img_size=(224,224)):
    rows = len(imgs)
    fig = plt.figure(figsize=(cols*size, rows*size))
    for i in range(rows):
        fig.add_subplot(rows, cols, i+1)
        plt.imshow(imgs[i])
    plt.suptitle(title)
    plt.show()
    

    
def resample(image, scan, new_spacing=[1,1,1]):
    # Determine current pixel spacing
    spacing       = np.array([scan[0].SliceThickness] + list(scan[0].PixelSpacing), dtype=np.float32)
    resize_factor = spacing / new_spacing
    new_shape     = np.round(image.shape * resize_factor)
    
    # recompute the resize factor and spacing such that we match the rounded new shape above
    rounded_resize_factor = new_shape / image.shape
    rounded_new_spacing   = spacing / rounded_resize_factor
    print(rounded_resize_factor)
    # zoom with resize factor
    image = scipy.ndimage.interpolation.zoom(image, (1,1,rounded_resize_factor[2]), mode='nearest')
    
    return image, rounded_new_spacing



def load_scans(dcm_path):
    
    slices = [pydicom.dcmread(file) for file in glob.glob(dcm_path+"/*")]
    slices.sort(key = lambda x: float(x.ImagePositionPatient[2]))
#     else: #basepath == "../input/osic-pulmonary-fibrosis-progression/":    
#         # This competition shows missing values in ImagePosition,
#         # this is why we are sorting by filename number
#         files = listdir(dcm_path)
#         file_nums = [np.int(file.split(".")[0]) for file in files]
#         sorted_file_nums = np.sort(file_nums)[::-1]
#        slices = [pydicom.dcmread(dcm_path + "/" + str(file_num) + ".dcm" ) for file_num in sorted_file_nums]
    return slices




path2 = "../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00018/FLAIR/"
#path2      = "../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00122/T1wCE/"


boxtest1   = load_FULL_brain(path2)
scans      = load_scans(path2)

print(boxtest1.shape)

#get a volume for 3d cnn
vol = boxtest1[:,:,60:60+32]
print(vol.shape)
plot_imgs(vol)
t0 = time.time()
img_resampled, spacing = resample(boxtest1, scans, [1,1,1])
t1 = time.time()
print(t1-t0)
print(img_resampled.shape)
plot_imgs(img_resampled)

#boxtest2 = load_FULL_brain_restricted(path2)
#print(boxtest2.shape)

#plot_imgs(boxtest2[:,:,-30:-10])

In [ ]:

def get_image_plane(data):
    '''
    Returns the MRI's plane from the dicom data.
    
    '''
    x1,y1,_,x2,y2,_ = [round(j) for j in data.ImageOrientationPatient]
    cords = [x1,y1,x2,y2]

    if cords == [1,0,0,0]:
        return 'coronal'
    if cords == [1,0,0,1]:
        return 'axial'
    if cords == [0,1,0,0]:
        return 'sagittal'
    
def get_image(data):
    '''
    Returns the image data as a numpy array.
    '''  
    if np.max(data.pixel_array)==0:
        img = data.pixel_array
    else:
        img = data.pixel_array/np.max(data.pixel_array)
        img = (img * 255).astype(np.uint8)
        
    return img

def sorted_image_dirs(path: str):
    '''
    Sorts the list of image directories by image number in a path
    '''
    dirs = glob.glob(path+'*')
    dirs.sort(key=lambda x: int(x.split('/')[-1].split('-')[-1].split('.')[0]))
    
    return dirs


def get_all_images(path: str):
    '''
    Returns a list of (non blank) images from a given path (of shape [non_blank_image_count, 512, 512])
    '''
    image_dirs = sorted_image_dirs(path)
    images = []
    
    for directory in image_dirs:
        data = pydicom.dcmread(directory)
        
        img = get_image(data)
        
        # Exclude the blank images
        if np.max(img)!=0:
            images.append(img)
        else:
            pass
    
    return images

def get_plane(path: str, plane=False):
    '''
    Returns the middle image from the path
    
    if plane=True returns the plane
    '''
    image_dirs = sorted_image_dirs(path)

    dicom = pydicom.dcmread(image_dirs[len(image_dirs)//2])
    img   = get_image(dicom)
    plane = get_image_plane(dicom)
  
    return plane



In [ ]:
seq_axes = {'flair':('flair_axis', []), 't1w':('t1w_axis', []), 't1wce':('t1wce_axis',[]), 't2w':('t2w_axis',[])}

for key, value in seq_axes.items():
    df_train[value[0]] = df_train[key].apply(lambda dicom_dir : get_plane(dicom_dir, plane=True))


In [ ]:
# Exclude bad samples
to_exclude = ["00109", "00123", "00709"]
print(df_train.shape[0])
df_train = df_train[~df_train['PatientID'].isin(to_exclude)]
print(df_train.shape[0])

df_train.head()

# Create TEST dataset

In [ ]:
df_test = df_train.sample(frac=0.17, random_state=2022)

In [ ]:
print(len(df_train.index))

print(len(df_test.index))

sns.countplot(data=df_test, x='MGMT_value')

In [ ]:
patient = "00811"
slices_path = df_train['flair'][df_train['PatientID'] == patient ].iloc[0]
print(slices_path)
flair_images = load_FULL_brain_restricted(slices_path)

print('No of images:', flair_images.shape[2])
print('MGMT: ', df_train['MGMT_value'][df_train['PatientID'] == patient ].iloc[0])
print(flair_images.shape)
# np_images =  np.array([im for im in list(flair_images.values())]).T

# Expand the dimensions of the image, this is so that it fits the expected model input format
# The shape of the input should be (batch_size, width, height, nr_channels) = (1, 224, 224, 3)
imgs_expanded = [np.expand_dims(flair_images[:,:,idx], axis = 2) for idx  in range(flair_images.shape[2])]
imgs_tensor   = [tf.convert_to_tensor(img) for img in imgs_expanded]
rgb_converted = [tf.image.grayscale_to_rgb(img) for img in imgs_tensor]

# rgb_converted = tf.image.grayscale_to_rgb(img_tensor)

img_expanded = np.expand_dims(rgb_converted[12], axis = 0)
print("Input shape:",img_expanded.shape)
fig, ax = plt.subplots(1,2, figsize=(30,10))

index = -2
ax[0].imshow(rgb_converted[index], cmap='icefire')
ax[1].imshow(flair_images[:,:,index], cmap='icefire')

# Pre-process the img in the same way original images were
img_ready = preprocess_input(img_expanded)
print("Input shape <after preproc>:", img_ready.shape)
# result = embedding_model.predict(img_ready)
# print("Output shape:", result.shape)

In [ ]:
# Extract FLAIR 
df_flair_mgmt_on  = df_train[df_train["MGMT_value"]== 1][['PatientID', 'flair', 'flair_axis']]
df_flair_mgmt_off = df_train[df_train["MGMT_value"]== 0][['PatientID', 'flair', 'flair_axis']]

# Extract t1w 
df_t1w_mgmt_on  = df_train[df_train["MGMT_value"]== 1][['PatientID', 't1w', 't1w_axis']]
df_t1w_mgmt_off = df_train[df_train["MGMT_value"]== 0][['PatientID', 't1w', 't1w_axis']]

# Extract t1wce
df_t1wce_mgmt_on  = df_train[df_train["MGMT_value"]== 1][['PatientID', 't1wce', 't1wce_axis']]
df_t1wce_mgmt_off = df_train[df_train["MGMT_value"]== 0][['PatientID', 't1wce', 't1wce_axis']]

# Extract t2w
df_t2w_mgmt_on  = df_train[df_train["MGMT_value"]== 1][['PatientID', 't2w', 't2w_axis']]
df_t2w_mgmt_off = df_train[df_train["MGMT_value"]== 0][['PatientID', 't2w', 't2w_axis']]

# Test slicing
df_flair_mgmt_on.head()

In [ ]:
# Cleaning up
df_flair_mgmt_on  = df_flair_mgmt_on.rename(columns={"flair_axis": "axis", 'flair': 'slices'})
df_flair_mgmt_off = df_flair_mgmt_off.rename(columns={"flair_axis": "axis", 'flair': 'slices'})

df_t1w_mgmt_on  = df_t1w_mgmt_on.rename(columns={"t1w_axis": "axis", 't1w': 'slices'})
df_t1w_mgmt_off = df_t1w_mgmt_off.rename(columns={"t1w_axis": "axis",'t1w': 'slices'})

df_t2w_mgmt_on  = df_t2w_mgmt_on.rename(columns={"t2w_axis": "axis", 't2w': 'slices'})
df_t2w_mgmt_off = df_t2w_mgmt_off.rename(columns={"t2w_axis": "axis", 't2w': 'slices'})

df_t1wce_mgmt_on  = df_t1wce_mgmt_on.rename(columns={"t1wce_axis": "axis", 't1wce': 'slices'})
df_t1wce_mgmt_off = df_t1wce_mgmt_off.rename(columns={"t1wce_axis": "axis",  't1wce': 'slices'})

In [ ]:
fig, ax = plt.subplots(1,4, figsize=(30,10))

sns.countplot(x='axis', data=df_flair_mgmt_on, ax=ax[0])
sns.countplot(x='axis', data=df_t1w_mgmt_on, ax=ax[1])
sns.countplot(x='axis', data=df_t2w_mgmt_on, ax=ax[2])
sns.countplot(x='axis', data=df_t1wce_mgmt_on, ax=ax[3])
plt.show()

fig, ax = plt.subplots(1,4, figsize=(30,10))
sns.countplot(x='axis', data=df_flair_mgmt_off, ax=ax[0])
sns.countplot(x='axis', data=df_t1w_mgmt_off, ax=ax[1])
sns.countplot(x='axis', data=df_t2w_mgmt_off, ax=ax[2])
sns.countplot(x='axis', data=df_t1wce_mgmt_off, ax=ax[3])

plt.show()

In [ ]:
df_train["flair_arr"] = df_train['flair'].apply(lambda slices_path : get_ordered_slices(slices_path))
df_train["t1w_arr"]   = df_train['t1w'].apply(lambda slices_path : get_ordered_slices(slices_path))
df_train["t1wce_arr"] = df_train['t1wce'].apply(lambda slices_path : get_ordered_slices(slices_path))
df_train["t2w_arr"]   = df_train['t2w'].apply(lambda slices_path : get_ordered_slices(slices_path))

In [ ]:
entry = 11
print(df_train["flair"].iloc[entry])

print(len(df_train["flair_arr"].iloc[entry]))
print(len(df_train["t1w_arr"].iloc[entry]))
print(len(df_train["t1wce_arr"].iloc[entry]))
print(len(df_train["t2w_arr"].iloc[entry]))


def get_len(data):
    lens = []
    for xi in data:
        lens.append(len(xi))
    return np.array(lens)
    
    
def plot_hist(np_arr,axd):
    sns.histplot(np_arr, bins = [0,50,100,150,200,250, 300, 350, 400, 450, 500, 550, 600], ax=axd)
    

df_train.head()
# 240
# 32
# 360
# 288


fig, ax   = plt.subplots(1,4, figsize=(30,10))
flair_arr = get_len(np.array(df_train["flair_arr"]))
t1w_arr   = get_len(np.array(df_train["t1w_arr"]))
t2w_arr   = get_len(np.array(df_train["t2w_arr"]))
t1wce_arr = get_len(np.array(df_train["t1wce_arr"]))

plot_hist(flair_arr, ax[0])
plot_hist(t1w_arr, ax[1])
plot_hist(t1wce_arr, ax[2])
plot_hist(t2w_arr, ax[3])

plt.show()

In [ ]:
df_train.head(15)

In [ ]:
# generate training images
# an image contain all 4 modalities disposed in a square
# df_mgmt_on  = [df_flair_mgmt_on,
#             df_t1w_mgmt_on,
#             df_t1wce_mgmt_on,
#             df_t2w_mgmt_on,
#            ]

# df_mgmt_off = [df_flair_mgmt_off, 
#             df_t1w_mgmt_off, 
#             df_t1wce_mgmt_off, 
#             df_t2w_mgmt_off 
#            ]


In [ ]:
def _circumscriber(img: np.array) -> np.array:
    #First is vertical, second is horizontal, third is slices
    vmin = 0
    vlimit = img.shape[0]
    hmin = 0
    hlimit = img.shape[1]
    
    for i in range(vlimit):
        if np.max(img[i, :]) == 0:
            vmin += 1
        else:
            break
    vmax = vmin + 1
    for i in range(vmin+1, vlimit):
        if np.max(img[i, :]) > 0:
            vmax += 1
        else:
            break
    
    for j in range(hlimit):
        if np.max(img[:, j]) == 0:
            hmin += 1
        else:
            break
    hmax = hmin + 1
    for j in range(hmin+1, hlimit):
        if np.max(img[:, j]) > 0:
            hmax += 1
        else:
            break
    return img[vmin:vmax, hmin:hmax]



def load_slices_meta(path: str):
    # load the DICOM files
    images = [os.path.basename(f) for f in glob.glob(f'{path}/*.dcm')]
    # images.sort(key=lambda f: int(re.sub('\D', '', f)))
    images = [pydicom.read_file("{}/{}".format(path,f)) for f in images]
    slices = sorted(images, key=lambda s: s.SliceLocation)
    try:
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
        
    for s in slices:
        s.SliceThickness = slice_thickness
  
    return slices


    
def _dicom_meta2array(dicom, voi_lut=True, fix_monochrome=True, resize=True, img_size=512):
    data  = apply_voi_lut(dicom.pixel_array, dicom)
    #data = dicom.pixel_array
    if dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
    #Normalize the data: subtract off the minimum, divide by the maximum, convert to 256 uint8
    data = data - np.min(data)
    data = data/np.max(data)
    data = (data * 255).astype(np.uint8)
    # Resize images to target value
    data = cv2.resize(data, (img_size, img_size))
    data = cv2.equalizeHist(data)
    return data



def reslice(slices):
    # pixel aspects, assuming all slices are the same
    # ps = slices[0].PixelSpacing
    # ss = slices[0].SliceThickness
    # ax_aspect  = ps[1]/ps[0]
    # sag_aspect = ps[1]/ss
    # cor_aspect = ss/ps[0]

    # create 3D array
    img_shape = list(_dicom_meta2array(slices[0]).shape)
    img_shape.append(len(slices))
    img3d = np.zeros(img_shape)

    # fill 3D array with the images from the files
    for i, s in enumerate(slices):
        img2d = _dicom_meta2array(s)
        img3d[:,:,i] = img2d
    
    print(img3d.shape)
    # img3d = _circumscriber(img3d)
    # print(img3d.shape)
    
    # plot 3 orthogonal slices
    fig, ax = plt.subplots(1,3, figsize=(30,10))
    
    p_slice = img3d[:,:,img_shape[2]//3]
    #p_slice = _circumscriber(p_slice)
    print(p_slice.shape)
    ax[0].imshow(p_slice)
    #ax[0].set_aspect(ax_aspect)
    
    img_size = 512
    #sag_aspect = sag_aspect * (img_shape[0]/img_shape[2] + 2)
    p_slice = np.rot90(img3d[:,img_shape[1]//3,:])
    p_slice = cv2.resize(p_slice, (img_size, img_size))
    #p_slice = _circumscriber(p_slice)
    # shape of 3d: (512, 512, 352)
    ax[1].imshow(p_slice)
    # ax[1].set_aspect(sag_aspect)
    # if(len(slices) < 50):
    #    ax[1].set_aspect(sag_aspect*20)
    

    # cor_aspect = cor_aspect * (img_shape[2]/img_shape[0])
    p_slice = np.rot90(img3d[img_shape[0]//3,:,:])
    p_slice = cv2.resize(p_slice, (img_size, img_size))
    #p_slice = _circumscriber(p_slice)
    print(p_slice.shape)
    ax[2].imshow(p_slice)
    #ax[2].set_aspect(cor_aspect)
    #if(len(slices) < 50): 
    #    ax[2].set_aspect(cor_aspect*20)


slices_path_t1wce = "../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00014/T1wCE/"
slices_path_t1w   = "../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00014/T1w/"
slices_path_flair = "../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00014/FLAIR/"
slices_path_t2w   = "../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00014/T2w/"


slices_t1wce = load_slices_meta(slices_path_t1wce)
slices_flair = load_slices_meta(slices_path_flair)
slices_t1w   = load_slices_meta(slices_path_t1w)
slices_t2w   = load_slices_meta(slices_path_t2w)


print(len(slices_t1wce))
print(len(slices_flair))
print(len(slices_t1w))
print(len(slices_t2w))


# reslice(slices_t1wce)
# reslice(slices_t1w)
# reslice(slices_t2w)
# reslice(slices_flair)

In [ ]:
import scipy


def resample(slices, new_spacing=[1,1,1]):
    # Determine current pixel spacing
    new_spacing = map(float, (new_spacing))
    image = scipy.ndimage.interpolation.zoom(slices, new_spacing)
    return image
    
    
def reslice_hu(slices):
    # create 3D array
    img_shape = list(slices.T.shape)
    img_shape.append(len(slices))
    img3d=np.zeros(slices.T.shape)
    img_size = 512
    # fill 3D array with the images from the files
    for i, d_slice in enumerate(slices):
        #print(d_slice.shape)
        img3d[:,:,i] = d_slice

    # plot 3 orthogonal slices
    fig, ax = plt.subplots(1,3, figsize=(30,10))
    ax[0].imshow(img3d[:,:,img_shape[2]//3])
    
    p_slice = np.rot90(img3d[:,img_shape[1]//3,:])
    p_slice = cv2.resize(p_slice, (img_size, img_size))
    ax[1].imshow(p_slice)
   
    p_slice = np.rot90(img3d[img_shape[0]//3,:,:])
    p_slice = cv2.resize(p_slice, (img_size, img_size))
    ax[2].imshow(p_slice)

In [ ]:
patient = "00005"

slices_path_t1wce = df_train['t1wce'][df_train['PatientID'] == patient ].iloc[0]
print(slices_path_t1wce)
#flair_images = load_FULL_brain_restricted(slices_path)
#print(flair_images.shape)
#plt.imshow(flair_images[:,:,7])

slices_twice  = load_slices_meta(slices_path_t1wce)
slices_pixels = get_pixels_hu(slices_twice)

print(slices_pixels.shape)
#pix_resampled, spacing = resample(slices_pixels, slices, [4,1,1])
print(slices_pixels.shape)


# reslice(slices_twice)
# reslice_hu(slices_pixels)

path_00014 = "../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00014/FLAIR/"
path2 = "../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00122/T1wCE/"
#boxtest1   = load_FULL_brain(path_00014)
#print(boxtest1.shape)

#boxtest2 = load_FULL_brain_restricted(path2)
#print(boxtest2.shape)

#plot_imgs(boxtest2[:,:,-30:-10])
df_train.head()


# Create model

In [ ]:
def create_3d_conv_layer(input_mod):
    x = Conv3D(filters=32, kernel_size=3, activation="relu",padding='same')(input_mod)
    x = Conv3D(filters=32, kernel_size=3, activation="relu",padding='same')(x)
    x = MaxPool3D(pool_size=2)(x)
    x = BatchNormalization()(x)

    x = Conv3D(filters=64, kernel_size=3, activation="relu")(x)
    x = Conv3D(filters=64, kernel_size=3, activation="relu")(x)
    x = MaxPool3D(pool_size=2)(x)
    x = BatchNormalization()(x)

    x = Conv3D(filters=128, kernel_size=3, activation="relu")(x)
    x = Conv3D(filters=128, kernel_size=3, activation="relu")(x)
    x = MaxPool3D(pool_size=2)(x)
    x = BatchNormalization()(x)

    x = Conv3D(filters=256, kernel_size=3, activation="relu", padding='same')(x)
    x = MaxPool3D(pool_size=2, padding='same')(x)
    x = BatchNormalization()(x)
     
    
    x = GlobalAveragePooling3D()(x)
    x = Dense(units=512, activation="relu")(x)
    x = Dropout(0.3)(x)
    
    return x

    
    
def get_model_3d(width=256, height=256, depth=32):
    """Build a 3D convolutional neural network model."""

    # inputs = keras.Input((width, height, depth, 1))
    input_flair = Input((width, height, depth, 1))
    input_t1w   = Input((width, height, depth, 1))
    input_t1wce = Input((width, height, depth, 1))
    input_t2w   = Input((width, height, depth, 1))
    
    # models
    mod_flair = create_3d_conv_layer(input_flair)
    mod_t1w   = create_3d_conv_layer(input_t1w)
    mod_t1wce = create_3d_conv_layer(input_t1wce)    
    mod_t2w   = create_3d_conv_layer(input_t2w)
    
    conv = concatenate([mod_flair, mod_t1w, mod_t1wce, mod_t2w])
    
    conv   = Flatten()(conv)
    dense  = Dense(512)(conv)
    dense  = LeakyReLU(alpha=0.1)(dense)
    dense  = Dropout(0.5)(dense)
    output = Dense(units=1, activation='sigmoid')(dense)

    model  = Model(inputs=[input_flair, input_t1w, input_t1wce, input_t2w], outputs=output, name="3dcnn")
    return model



from cloud_tpu_client import Client
# c = Client()
# c.configure_tpu_version(tf.__version__, restart_type="always")

#detect and init the TPU
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
print("Running on TPU ", tpu.cluster_spec().as_dict()["worker"])

# #instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)
with tpu_strategy.scope():
    # Build model.
    model = get_model(width=256, height=256, depth=32)
    # Compile model.
    initial_learning_rate = 0.0001
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True
    )

    model.compile(
        loss="binary_crossentropy",
        optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
        metrics=["acc"],
    )
    
    
model.summary()
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
def create_3d_conv_layer(input_mod):
    x = Conv3D(filters=32, kernel_size=3, activation="relu",padding='same')(input_mod)
    x = Conv3D(filters=32, kernel_size=3, activation="relu",padding='same')(x)
    x = MaxPool3D(pool_size=2)(x)
    x = BatchNormalization()(x)

    x = Conv3D(filters=64, kernel_size=3, activation="relu")(x)
    x = Conv3D(filters=64, kernel_size=3, activation="relu")(x)
    x = MaxPool3D(pool_size=2)(x)
    x = BatchNormalization()(x)

    x = Conv3D(filters=128, kernel_size=3, activation="relu")(x)
    x = Conv3D(filters=128, kernel_size=3, activation="relu")(x)
    x = MaxPool3D(pool_size=2)(x)
    x = BatchNormalization()(x)

    x = Conv3D(filters=256, kernel_size=3, activation="relu", padding='same')(x)
    x = MaxPool3D(pool_size=2, padding='same')(x)
    x = BatchNormalization()(x)
     
    
    x = GlobalAveragePooling3D()(x)
    x = Dense(units=512, activation="relu")(x)
    x = Dropout(0.3)(x)
    
    return x

    
    
def get_model_3d(width=256, height=256, depth=32):
    """Build a 3D convolutional neural network model."""

    # inputs = keras.Input((width, height, depth, 1))
    input_flair = Input((width, height, depth, 1))
    input_t1w   = Input((width, height, depth, 1))
    input_t1wce = Input((width, height, depth, 1))
    input_t2w   = Input((width, height, depth, 1))
    
    # models
    mod_flair = create_3d_conv_layer(input_flair)
    mod_t1w   = create_3d_conv_layer(input_t1w)
    mod_t1wce = create_3d_conv_layer(input_t1wce)    
    mod_t2w   = create_3d_conv_layer(input_t2w)
    
    conv = concatenate([mod_flair, mod_t1w, mod_t1wce, mod_t2w])
    
    conv   = Flatten()(conv)
    dense  = Dense(512)(conv)
    dense  = LeakyReLU(alpha=0.1)(dense)
    dense  = Dropout(0.5)(dense)
    output = Dense(units=1, activation='sigmoid')(dense)

    model  = Model(inputs=[input_flair, input_t1w, input_t1wce, input_t2w], outputs=output, name="3dcnn")
    return model



from cloud_tpu_client import Client
# c = Client()
# c.configure_tpu_version(tf.__version__, restart_type="always")

#detect and init the TPU
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
print("Running on TPU ", tpu.cluster_spec().as_dict()["worker"])

# #instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)
with tpu_strategy.scope():
    # Build model.
    model = get_model(width=256, height=256, depth=32)
    # Compile model.
    initial_learning_rate = 0.0001
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True
    )

    model.compile(
        loss="binary_crossentropy",
        optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
        metrics=["acc"],
    )
    
    
model.summary()
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
#print(df_train.head())
print(len(df_train.index))
df_train[(df_train['flair_axis'] == "axial") |  (df_train['t1w_axis'] == "axial") |  (df_train['t2w_axis'] == "axial") |  (df_train['t1wce_axis'] == "axial")]


In [ ]:
path_00014 = "../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00014/FLAIR/"
#boxtest1   = load_FULL_brain(path_00014)
#print(boxtest1.shape)

t0= time.time()
boxtest2 = load_FULL_brain_restricted(path_00014)
t1 = time.time()
print(t1-t0)
# returns  (512, 512, 82)

In [ ]:
entry = df_train.sample(1).iloc[0]

entry["flair_axis"]

In [ ]:
modalities = { 0 : "flair",
               1 : "t1w",
               2 : "t1wce",
               3 : "t2w"
             }

def train_preprocessing(volume, label):
    """Process training data by rotating and adding a channel."""
    # Rotate volume
    #volume = rotate(volume)
    volume = tf.expand_dims(volume, axis=3)
    return volume, label


def validation_preprocessing(volume, label):
    """Process validation data by only adding a channel."""
    volume = tf.expand_dims(volume, axis=3)
    return volume, label


def get_ordered_slices_subset(path:str, proportion:float, slices_count:int):
    """
    get a list of ordered slices
    """
    images = [os.path.basename(f) for f in glob.glob(f'{path}/*.dcm')]
    images.sort(key=lambda f: int(re.sub('\D', '', f)))
    
    len_imgs = len(images)
    fifth    = len_imgs//5
    # Most of images have a lot of empty space, get rid of 2/5 of them
    images   = images[fifth:-fifth]
    len_imgs = len(images)
    
    if len_imgs < slices_count:
        return None
    elif len_imgs == slices_count:
        return images
    
    index_start = int(proportion * len_imgs)                   
    if index_start + slices_count > len_imgs:
        subset = images[-slices_count:]
    else:
        subset = images[index_start:index_start+slices_count]
        
    return subset


def is_bad_sample(selection):
    bad_slices = ["00584/T1w", "00569/T2w", '00998/T1wCE', '00584/T1w', '00137/T1w', '00998/T2w', '00589/T1w' ]
    for badness in bad_slices:
            if (badness in selection):
                return 1
    return 0


def load_3d_volume(df, slices_count=32):
    """
    send all of the images in the chosen modality, in order, as a single 3D np array
    """
    coin  = np.random.choice([0,1])
    proportion = random.random()
    #modality   = np.random.choice(list(modalities.values()))
    
    
    patient  = patients.sample(1).iloc[0]
    mgmt_val = patient["MGMT_value"].iloc[0]
    
    dcm_path_flair = patient[modalities[0]]
    dcm_path_t1w   = patient[modalities[1]]
    dcm_path_t1wce = patient[modalities[2]]
    dcm_path_t2w   = patient[modalities[3]]
    
    images_flair = get_ordered_slices_subset(dcm_path_flair, proportion, slices_count)
    images_t1w   = get_ordered_slices_subset(dcm_path_t1w, proportion, slices_count)
    images_t1wce = get_ordered_slices_subset(dcm_path_t1wce, proportion, slices_count)
    images_t2w   = get_ordered_slices_subset(dcm_path_t2w, proportion, slices_count)
    
    while images_flair == None or images_t1w == None or images_t1wce == None or images_t2w == None:
        patient  = patients.sample(1).iloc[0]
        mgmt_val = patient["MGMT_value"].iloc[0]
        
        dcm_path_flair = patient[modalities[0]]
        dcm_path_t1w   = patient[modalities[1]]
        dcm_path_t1wce = patient[modalities[2]]
        dcm_path_t2w   = patient[modalities[3]]

        images_flair = get_ordered_slices_subset(dcm_path_flair, proportion, slices_count)
        images_t1w   = get_ordered_slices_subset(dcm_path_t1w, proportion, slices_count)
        images_t1wce = get_ordered_slices_subset(dcm_path_t1wce, proportion, slices_count)
        images_t2w   = get_ordered_slices_subset(dcm_path_t2w, proportion, slices_count)
        
    
    # Load meta from files
    pixels_flair = [_dicom2array(dcm_path_flair + f) for f in images_flair]
    pixels_t1w   = [_dicom2array(dcm_path_t1w + f) for f in images_t1w]
    pixels_t1wce = [_dicom2array(dcm_path_t1wce + f) for f in images_t1wce]
    pixels_t2w   = [_dicom2array(dcm_path_t2w + f) for f in images_t2w]
    
    # np.count_nonzero(anchor) < 10000)
    vol_flair = np.array([im for im in pixels_flair]).T
    vol_flair = tf.convert_to_tensor(tf.expand_dims(vol_flair, axis=3))
    
    vol_t1w   = np.array([im for im in pixels_t1w]).T
    vol_t1w   = tf.convert_to_tensor(tf.expand_dims(vol_t1w, axis=3))
    
    vol_t1wce = np.array([im for im in pixels_t1wce]).T
    vol_t1wce = tf.convert_to_tensor(tf.expand_dims(vol_t1wce, axis=3))
    
    vol_t2w   = np.array([im for im in pixels_t2w]).T
    vol_t2w   = tf.convert_to_tensor(tf.expand_dims(vol_t2w, axis=3))
    
    
    #final_image = _circumscriber(good_images)
    volumes = [vol_flair,vol_t1w, vol_t1wce, vol_t2w]
    return volumes,  mgmt_val


def generate_batches(df, total_items):
    #patients_axial = df_train[(df_train['flair_axis'] == "axial") &  (df_train['t1w_axis'] == "axial") &  (df_train['t2w_axis'] == "axial") &  (df_train['t1wce_axis'] == "axial")]
    slices_count = 32
    i = 0
    while True:
        volumes, label = load_3d_volume(df, slices_count)
        yield volumes, tf.convert_to_tensor(label)
        #i += 1
    

    
def generate_batches_2(df, total_items):
    #patients_axial = df_train[(df_train['flair_axis'] == "axial") &  (df_train['t1w_axis'] == "axial") &  (df_train['t2w_axis'] == "axial") &  (df_train['t1wce_axis'] == "axial")]
    slices_count = 32
    i            = 0
    flair  = np.zeros((total_items, 256, 256, slices_count, 1))
    t1w    = np.zeros((total_items, 256, 256, slices_count, 1))
    t1wce  = np.zeros((total_items, 256, 256, slices_count, 1))
    t2w    = np.zeros((total_items, 256, 256, slices_count, 1))
    labels = np.zeros((total_items, ))
    
    for i in range(total_items):
        volumes, label = load_3d_volume(df, slices_count)
        flair[i]  = volumes[0]
        t1w[i]    = volumes[1]
        t1wce[i]  = volumes[2]
        t2w[i]    = volumes[3]
        labels[i] = label
    return [flair, t1w, t1wce, t2w], labels

        
# TEST        
patient_entry = df_train.iloc[1]
axis     = "flair_axis"
modality = "flair"

t0 = time.time()
sample_generator = generate_batches_2(df_test, 4)
sample_test = sample_generator
t1 = time.time()
print(t1-t0)
print("--------")

print(len(sample_test[0]))
print(len(sample_test))

print(sample_test[0][0].shape)
print(sample_test[0][1].shape)
print(sample_test[0][2].shape)
print(sample_test[0][3].shape)


print(sample_test[1].shape)
#print(sample_test[0])
print(sample_test[1])



In [ ]:
# TEST  
x = model.predict(sample_test[0])


In [ ]:
#train_generator = tf.data.Dataset.from_generator(generate_batches_train, output_types={ "slices":tf.uint8, "cls_lbl":tf.uint8})
# validation_generator = tf.data.Dataset.from_generator(generate_batches_test, output_types={ "slices":tf.uint8, "cls_lbl":tf.uint8})

# train_generator      = next(generate_batches_train())
# validation_generator = next(generate_batches_test()) 


#@tf.function
def input_fn(df, batch_size, epochs):
    features_shape = [256, 256, 32, 1]
    labels_shape   = [] 
    dataset = tf.data.Dataset.from_generator(lambda: generate_batches(df, batch_size),
                                             output_types=((tf.uint8,
                                                            tf.uint8,
                                                            tf.uint8,
                                                            tf.uint8), tf.int64),
                                             output_shapes=((tf.TensorShape(features_shape),
                                                             tf.TensorShape(features_shape),
                                                             tf.TensorShape(features_shape),
                                                             tf.TensorShape(features_shape)), tf.TensorShape(labels_shape))
                                            )
                                            

    #dataset = dataset.repeat(epochs)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(2)
    return dataset



def input_fn_2(df, batch_size):
    sample_generator = generate_batches_2(df, batch_size)
                                    
    dataset = tf.data.Dataset.from_tensor_slices(sample_generator)
    #dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(2)
    return dataset


# TEST
# batch_size = 8
# epochs    = 10
# train_generator = input_fn_2(df_train, batch_size)
# data = train_generator.take(1)
# images, labels = list(data)[0]
# print(labels)
# print(images.shape)

# for next_element in train_generator:
#     tf.print(next_element[1].shape)
#     tf.print(next_element[0].shape)    

In [ ]:

# Define callbacks.
load_locally = tf.saved_model.LoadOptions(experimental_io_device='/job:localhost')
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    "3d_image_classification.h5", save_best_only=True
)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor="val_acc", patience=15)


# Train the model, doing validation at the end of each epoch

epochs      = 10
batch_size  = 16
num_batches = len(df_train.index)//batch_size * 2

# epochs = epochs,
# steps_per_epoch = num_batches,
# train_generator      = input_fn(df_train, batch_size, epochs)
# validation_generator = input_fn(df_test, batch_size, epochs)


#_train_generator      = generate_batches_2(df_train, batch_size)
#_validation_generator = generate_batches_2(df_test, batch_size)
#print(_train_generator)

# model.fit(_train_generator,
#     validation_data=_validation_generator,
#     epochs=epochs,
#     steps_per_epoch = num_batches,
#     validation_steps = num_batches,
#     shuffle=True,
#     verbose=1,
#     callbacks=[checkpoint_cb,early_stopping_cb])
losses     = []
val_losses = []
stop = 5
for i in tqdm(range(10000)):
    X_train, y_train = generate_batches_2(df_train, batch_size)
    X_test, y_test   = generate_batches_2(df_test, batch_size)
    
    h_callback = model.fit(X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=epochs,
        shuffle=True,
        verbose=1,
        callbacks=[checkpoint_cb,early_stopping_cb])
    losses     = losses     + h_callback.history['loss']
    val_losses = val_losses + h_callback.history['val_loss']
    if i == stop:
        break
    
    
    
# 1
# 144/144 [==============================] -                loss: 0.6855 - acc: 0.5620 - val_loss: 0.6783 - val_acc: 0.5894
# 144/144 [==============================] - 728s 5s/step - loss: 0.6856 - acc: 0.5675 - val_loss: 0.9366 - val_acc: 0.5043
# 144/144 [==============================] - 729s 5s/step - loss: 0.6770 - acc: 0.5692 - val_loss: 0.7183 - val_acc: 0.5694
# 144/144 [==============================] - 720s 5s/step - loss: 0.6839 - acc: 0.5593 - val_loss: 0.6938 - val_acc: 0.5495
# 144/144 [==============================] - 715s 5s/step - loss: 0.6919 - acc: 0.5324 - val_loss: 0.6973 - val_acc: 0.5000
# 144/144 [==============================] - 730s 5s/step - loss: 0.6812 - acc: 0.5609 - val_loss: 0.7013 - val_acc: 0.5582


#2

In [ ]:
print( h_callback.history['loss'])

plt.plot(losses)
plt.xlabel('Epochs')
plt.ylabel('Loss')
